In [1]:
import time
from datetime import datetime

import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score, explained_variance_score
from sklearn.metrics import mean_squared_log_error, mean_poisson_deviance, mean_gamma_deviance, mean_tweedie_deviance, max_error

%matplotlib inline

df = pd.read_csv("data/1_data_test_XOM_Q7R_TAH_0.csv")



In [2]:
df[df['Date'] == '2021-05-10']

,Date,Close,High,Low,Open,Volume,target,ema_5_Close,pred,error,bool,p_bool,right,name,target_name,days_ahead,day_predict


In [ ]:
plt.figure(figsize=(13,7))
plt.xticks(rotation=45) 

def plot_graph_market():
    for idx, val in df.iterrows():
        plt.plot([idx, idx], [val['Low'], val['High']], color='Black')


def plot_graph_real(col, color):
    plt.plot(df.index, df[col], color=color)
    plt.scatter(df.index, df[col], color=color)

def plot_graph_pred(index):
    x, y = [], []
    target = []
    color = []
    df_1 = df[df['days_ahead'] == index]
    [y.append(df_1['day_predict'].values[0][:10]) for i in range(3)]
    for i, j in zip(['red', 'blue', 'green'], range(3)):
        target.append(df_1['target'].values[j])
        x.append(df_1['pred'].values[j])
        color.append(i)
    plt.scatter(y, x, c=color, linewidths=4)
    # plt.scatter(y, target, c=color, linewidths=1)
    
targets_name = df['target_name'].drop_duplicates().values
colors = ['red', 'blue', 'green']

plot_graph_market()
plot_graph_real('Close', 'Black')
[plot_graph_real(i, j) for i, j in zip(targets_name, colors)]
[plot_graph_pred(i + 1) for i in range(4)]

plt.grid(True)
plt.show()


In [48]:
best = []

date = df['Date'].drop_duplicates()
for j in range(len(date)):
    ax_df = df[df['Date'] == date[j]]
    ax_best = ax_df.sort_values(['days_ahead', 'error'])
    for i in range(0, 11, 3):
        best.append(ax_best.iloc[i:i+1, 13:14].values[0][0][5])

best = pd.Series(best)
print(best.value_counts(normalize=True))
print(best.value_counts())